## Project: Nefret Söylemi Tespiti (Hate Speech Detection)

Nefret söylemi algılama modeli için kullanacağım veri seti bir test ve eğitim setinden oluşur. Eğitim paketi 31.962 tweet'lik bir liste, her tweet için karşılık gelen bir kimlik ve 0 veya 1 etiketi içerir. Bu veri setinde tespit etmemiz gereken belirli duygu, tweet'in nefret söylemine dayanıp dayanmadığıdır.

In [1]:
import pandas as pd
train = pd.read_csv('train.csv')
print("Training Set:"% train.columns, train.shape, len(train))
test = pd.read_csv('test.csv')
print("Test Set:"% test.columns, test.shape, len(test))

Training Set: (31962, 3) 31962
Test Set: (17197, 2) 17197


## Veri Temizleme

In [2]:
import re
def  clean_text(df, text_field):
    df[text_field] = df[text_field].str.lower()
    df[text_field] = df[text_field].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  
    return df
test_clean = clean_text(test, "tweet")
train_clean = clean_text(train, "tweet")

### Nefret Söylemi Tespit Modeli için Dengesiz Veriler Ele Alma

Aşırı örnekleme durumunda, azınlık sınıfından, sınıf çoğunluk ile aynı boyuta gelene kadar, tekrar tekrar, değiştirmeyle örnekleme yapan bir fonksiyon kullanırız. Bunu nasıl halledebileceğimize bakalım

In [3]:
from sklearn.utils import resample
train_majority = train_clean[train_clean.label==0]
train_minority = train_clean[train_clean.label==1]
train_minority_upsampled = resample(train_minority, 
                                 replace=True,    
                                 n_samples=len(train_majority),   
                                 random_state=123)
train_upsampled = pd.concat([train_minority_upsampled, train_majority])
train_upsampled['label'].value_counts()

label
1    29720
0    29720
Name: count, dtype: int64

## Bir Boru Hattı(Pipeline) Oluşturma

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
pipeline_sgd = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf',  TfidfTransformer()),
    ('nb', SGDClassifier()),])

## Nefret Söylemi Algılama Modelinin Eğitimi

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_upsampled['tweet'],
                                                    train_upsampled['label'],random_state = 0)

Şimdi modeli eğitelim ve F1 puan yöntemini kullanarak test setindeki sonuçları tahmin edelim

In [6]:
model = pipeline_sgd.fit(X_train, y_train)
y_predict = model.predict(X_test)
from sklearn.metrics import f1_score
f1_score(y_test, y_predict)

0.9695959461261502

## Sonuç:
* - Hate Speech Detection modelin performansı, veri seti ve modelin tipi ile bağlıdır.
* - Bu modelde, veri seti ve modelin tipi ile birlikte, performansı değerlendirilmelidir.
* - Modelin performansı, hate speech ve normal tweetlerin tahmin edilebilirliğini, doğruluğunu ve kesinlikle gerçek hate speech ve normal tweetlerin tahmin edilebilirliğini değerlendirmelidir.
